In [ ]:
from python_speech_features import mfcc
from CQCC.cqcc import cqcc
import scipy.io.wavfile as wav
import soundfile as sf
import os
import numpy as np
import pickle
import argparse
import pandas as pd
from embedding import *

parser = argparse.ArgumentParser()
parser.add_argument("--data_path", required=True, type=str,
                    help='path to data . For example,../Data/LA_CSV/train.csv')
parser.add_argument("--type", required=True, type=str, help='type of data :train,dev or eval')
args = parser.parse_args()

data = pd.read_csv(args.data_path)
feats = []
total_samples = len(data['filepath'])
progress_bar = tqdm(total=total_samples, bar_format='{l_bar}{bar:20}{r_bar}', desc=f"Preprocessing Audio",
                    unit="files")
for filepath in data['filepath']:
    signal, rate = sf.read(filepath[3:])
    mask = envelope(signal, rate, 0.0005)
    signal = signal[mask]
    feats_mfcc = compute_mfcc(signal[:2 * rate], rate)
    feats_mfcc = compute_feature_sequence_embedding(feats_mfcc, mean=True, variance=True, avg_diff=True)
    feats.append(feats_mfcc)
    progress_bar.update(1)
progress_bar.close()
feats = pd.DataFrame(feats)
feats['target'] = data['target'].values
if args.type == 'train':
    feats.to_csv('Data/features/' + args.type + 'feats.csv', index=False)
else:
    feats['system_id'] = data['system_id'].values
    feats.to_csv('Data/features/' + args.type + 'feats.csv', index=False)

In [ ]:
import pickle
import argparse
import pandas as pd
from sklearn.svm import SVC
from sklearn.mixture import GaussianMixture as GMM
from sklearn.mixture import BayesianGaussianMixture as DPGMM

parser = argparse.ArgumentParser()
parser.add_argument("--data_path", required=True, type=str, help='path to pickled file. For example, data/train.pkl')

args = parser.parse_args()

data = pd.read_csv(args.data_path)
y = data['target'].values
X = data.drop(['target'], axis=1).values

print('SVM training started...')
clf = SVC(class_weight='balanced')
clf.fit(X, y)
print('SVM training completed.')

with open('models/svm.pkl', 'wb') as outfile:
    pickle.dump(clf, outfile)

# Train GMM for class 0 (bon)
print('GMM training for class 0 (bon) started...')
gmm_bon = GMM(n_components=144, covariance_type='diag', n_init=50)
Xbon = data[data['target'] == 0].drop(['target'], axis=1).values
gmm_bon.fit(Xbon)
print('GMM training for class 0 (bon) completed.')

# Train GMM for class 1 (sp)
print('GMM training for class 1 (sp) started...')
gmm_sp = GMM(n_components=144, covariance_type='diag', n_init=50)
Xsp = data[data['target'] == 1].drop(['target'], axis=1).values
gmm_sp.fit(Xsp)
print('GMM training for class 1 (sp) completed.')

# Train DPGMM for class 0 (bon)
print('DPGMM training for class 0 (bon) started...')
dpgmm_bon = DPGMM(n_components=144, covariance_type='diag', n_init=50, weight_concentration_prior_type='dirichlet_process')
dpgmm_bon.fit(Xbon)
print('DPGMM training for class 0 (bon) completed.')

# Train DPGMM for class 1 (sp)
print('DPGMM training for class 1 (sp) started...')
dpgmm_sp = DPGMM(n_components=144, covariance_type='diag', n_init=50, weight_concentration_prior_type='dirichlet_process')
dpgmm_sp.fit(Xsp)
print('DPGMM training for class 1 (sp) completed.')

# Save models
pickle.dump(gmm_bon, open('models/bon_gmm.pkl', 'wb'))
pickle.dump(gmm_sp, open('models/sp_gmm.pkl', 'wb'))
pickle.dump(dpgmm_bon, open('models/bon_dpgmm.pkl', 'wb'))
pickle.dump(dpgmm_sp, open('models/sp_dpgmm.pkl', 'wb'))

print('GMM and DPGMM models created')